# Final Capstone Project

### Introduction
where you discuss the business problem and who would be interested in this project.

### Data
where you describe the data that will be used to solve the problem and the source of the data.

Two different data sets were used, both from the official Belgium Statistics Open Data repository (https://statbel.fgov.be/en/open-data?category=All)
- Statistical sectors 2020 https://statbel.fgov.be/en/open-data/statistical-sectors-2020 --> This is the minimum administrative sub-division 
- Population by statistical sector 2020 https://statbel.fgov.be/en/open-data/population-statistical-sector-8

At the start of this exercise, it is unclear if Population by statistical sector will be needed nevertheless it was decided to include it.

The data was prepared in QGIS and Excel as follows:
1. Change coordinate system to geographical coordinate system WGS 84 EPSG:4326, so the coordinates will come in decimal degrees, by means of Saving Layer As with different coordinate system
2. In Excel, save Population by statistical sector 2020 in CSV
3. Join attribute tables of imported CSV with those of the Statistical Sectors
4. Calculate centroids of Statistical Sectors, so we get points instead of polygons
6. Calculate geometry attributes of centroids
7. Create different data sets by cities of interest (Antwerpen, Leuven, Genk, Gent and Brussels region)

### Methodology
section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

In [281]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Antwerpen, Belgium'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Antwerpen, Belgium are 51.2211097, 4.3997081.


In [257]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

### Results
section where you discuss the results.

In [282]:
Antw = geopandas.read_file('./data/GIS files/NeighborhoodsAntwerpen.shp')
Antw.tail(1)

,CS01012020,T_SEC_NL,T_SEC_FR,T_SEC_DE,C_NIS6,T_NIS6_NL,T_NIS6_FR,CNIS5_2020,T_MUN_NL,T_MUN_FR,...,OPENDATA_2,OPENDATA_3,OPENDATA_4,OPENDATA_5,OPENDATA_6,OPENDATA_7,OPENDATA_8,lon,lat,geometry
298,11002V391,POLDERBOS,POLDERBOS,POLDERBOS,11002V,HOBOKEN,HOBOKEN,11002,Antwerpen,Anvers,...,Anvers,POLDERBOS,POLDERBOS,01Jan1983,31Dec9999,"116,7700119",2,4.340879,51.188705,POINT (4.34088 51.18870)


In [283]:
drop_columns = ['T_SEC_FR', 'T_SEC_DE', 'T_NIS6_FR', 'CNIS5_2020', 'T_MUN_FR', 'T_MUN_DE', 'C_ARRD',
       'T_ARRD_NL', 'T_ARRD_FR', 'T_ARRD_DE', 'C_PROVI', 'T_PROVI_NL',
       'T_PROVI_FR', 'T_PROVI_DE', 'C_REGIO', 'T_REGIO_NL', 'T_REGIO_FR',
       'T_REGIO_DE', 'C_COUNTRY', 'NUTS1', 'NUTS2', 'NUTS3', 'OPENDATA_S', 'OPENDATA_1', 'OPENDATA_2',
       'OPENDATA_3', 'OPENDATA_4', 'OPENDATA_5', 'OPENDATA_6', 'OPENDATA_7']
Antw.drop(columns = drop_columns, inplace=True)

In [284]:
Antw.tail(1)

,CS01012020,T_SEC_NL,C_NIS6,T_NIS6_NL,T_MUN_NL,M_AREA_HA,M_PERI_M,Density,OPENDATA_8,lon,lat,geometry
298,11002V391,POLDERBOS,11002V,HOBOKEN,Antwerpen,114.843019,8297.0,0,2,4.340879,51.188705,POINT (4.34088 51.18870)


In [285]:
rename_columns = ['Code1', 'Name', 'Code2', 'Name2', 'Municipality', 'Area_ha', 'Peri_m', 'Density', 'Population', 'Lon', 'Lat', 'Geometry']
Antw.columns = rename_columns
Antw.tail()

,Code1,Name,Code2,Name2,Municipality,Area_ha,Peri_m,Density,Population,Lon,Lat,Geometry
294,11002V301,MORETUSBURG,11002V,HOBOKEN,Antwerpen,21.409693,1962.0,82,1758,4.339631,51.171928,POINT (4.33963 51.17193)
295,11002V312,VISPUTTEN,11002V,HOBOKEN,Antwerpen,26.951873,2144.0,53,1431,4.340971,51.176224,POINT (4.34097 51.17622)
296,11002V322,POLDERSTAD,11002V,HOBOKEN,Antwerpen,117.529855,4651.0,14,1654,4.345302,51.187534,POINT (4.34530 51.18753)
297,11002V373,INDUSTRIEPARK,11002V,HOBOKEN,Antwerpen,63.371617,3946.0,0,17,4.336061,51.179483,POINT (4.33606 51.17948)
298,11002V391,POLDERBOS,11002V,HOBOKEN,Antwerpen,114.843019,8297.0,0,2,4.340879,51.188705,POINT (4.34088 51.18870)


# I had to bring them manually as the code is not working
latitude = 51.229539462642265
longitude = 4.404776373568066

In [286]:
import folium
# create map of Toronto using latitude and longitude values
map_city = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lon, Code1, Name in zip(Antw['Lat'], Antw['Lon'], Antw['Code1'], Antw['Name']):
    label = '{}, {}'.format(Code1, Name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_city)  
    
map_city

In [287]:
CLIENT_ID = 'G4CN4VUVBD0EFCCT0LXPFSLWKJAHXJT2BPMPOD235BCMQ1N4' # your Foursquare ID
CLIENT_SECRET = 'VFXVULBUNRTXAHD20FW3A3ZPMHU1V25BJPWLDLH5WDZF4BFB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [288]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [289]:
import requests
city_venues = getNearbyVenues(names=Antw['Name'],
                                   latitudes=Antw['Lat'],
                                   longitudes=Antw['Lon']
                                  )

STABROEK
ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN )
KLAPDORP - BROUWERSVLIET
GROENPLAATS (SPAANSE WALLEN)
HOOGSTRAAT (SPAANSE WALLEN)
OUDAAN (SPAANSE WALLEN)
GEVANGENIS (SPAANSE WALLEN)
SCHELDEKADEN NOORD
MEIR - LEYSSTRAAT (SP. WALLEN)
KIPDORP-ST.-JACOBS (SP.WALLEN)
K.N.S.-NAT. BANK (SP. WALLEN)
STADSWAAG-BEGIJNHOF(SP.WALLEN)
TABAKSVEST (SPAANSE WALLEN)
HESSENHUIS (SPAANSE WALLEN)
ST.-ANDRIES (SPAANSE WALLEN)
ST.-MICHIELSKAAI (SP. WALLEN)
LINKEROEVER-ZUID (LINKEROEVER)
LINKEROEVER - STATION
LINKEROEVER-NOORD
ST.-ANNA (LINKEROEVER)
THOENETLAAN (LINKEROEVER)
GLORIANTLAAN (LINKEROEVER)
GALGEWEEL (LINKEROEVER)
ST.-ANNABOS (LINKEROEVER )
BLANCEFLOERLAAN  (LINKEROEVER)
CHARLES DE COSTERLAAN(L.OEVER)
STATIEKWARTIER (STATIONSWIJK)
ATHENEUM (STATIONSWIJK)
DE CONINCPLEIN-Z.(STATIONSW.)
OFFERANDESTRAAT(STATIONSWIJK )
PROVINCIESTRAAT(STATIONSWIJK )
PELIKAANSTRAAT (STATIONSWIJK )
STATION - ZOO (STATIONSWIJK )
STADSPARK (STATIONSWIJK )
JEZUITENCOLLEGE(STATIONSWIJK )
DAMBRUGGESTRAAT-N.(STATIONSW.)


In [290]:
print(city_venues.shape)
city_venues.head()

(1712, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,STABROEK,51.329844,4.341878,kaai 734 zuidnatie,51.329007,4.343253,Port
1,STABROEK,51.329844,4.341878,Becomar Zuidnatie,51.330475,4.343896,Harbor / Marina
2,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN ),51.222503,4.400889,Grote Markt,51.221163,4.399810,Plaza
3,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN ),51.222503,4.400889,Bia Mara,51.220894,4.400189,Fish & Chips Shop
4,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN ),51.222503,4.400889,De Pottekijker,51.221097,4.401870,Belgian Restaurant


In [291]:
city_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
'T EILANDJE,18,18,18,18,18,18
'T FORT,3,3,3,3,3,3
'T LAAR,4,4,4,4,4,4
'T MESTPUTTEKE,7,7,7,7,7,7
ALBERTPARK (OOSTWIJK),8,8,8,8,8,8
...,...,...,...,...,...,...
ZANDVLIET-KERN(ZANDVL.-5 DIST),1,1,1,1,1,1
ZILVERBEEK,9,9,9,9,9,9
ZURENBORG,6,6,6,6,6,6


In [292]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))
print(city_venues['Venue Category'].unique())

There are 271 uniques categories.
['Port' 'Harbor / Marina' 'Plaza' 'Fish & Chips Shop' 'Belgian Restaurant'
 'Coffee Shop' 'French Restaurant' 'Bookstore' 'Italian Restaurant'
 'Steakhouse' 'Church' 'Restaurant' 'Fishing Store' 'Hotel'
 'Adult Boutique' 'History Museum' 'Gay Bar' 'Thai Restaurant'
 'Ethiopian Restaurant' 'Café' 'Soup Place' 'Empanada Restaurant' 'Bar'
 'American Restaurant' 'Health & Beauty Service' 'Nightclub' 'Waterfront'
 'Snack Place' 'Asian Restaurant' 'Greek Restaurant' 'Juice Bar'
 'Beer Bar' 'Frozen Yogurt Shop' 'Indonesian Restaurant' 'Cocktail Bar'
 'Deli / Bodega' 'Kitchen Supply Store' 'Clothing Store'
 'Vegetarian / Vegan Restaurant' 'Brasserie' 'Boutique' 'Gourmet Shop'
 'Sushi Restaurant' 'Perfume Shop' 'Burger Joint' 'Gaming Cafe'
 'Camera Store' 'Optical Shop' 'Cupcake Shop' 'Falafel Restaurant'
 'Pie Shop' 'Taco Place' 'Wine Bar' 'Ice Cream Shop' 'Art Museum'
 'Argentinian Restaurant' 'Comfort Food Restaurant' 'Breakfast Spot'
 'Mexican Restaurant' '

In [293]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
city_onehot['Neighborhood'] = city_venues['Neighborhood'] 

city_onehot.head()

,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Volleyball Court,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,STABROEK
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,STABROEK
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN )
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN )
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN )


In [294]:
# move neighborhood column to the first column
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Volleyball Court,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,STABROEK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,STABROEK,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN ),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN ),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN ),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [295]:
city_grouped = city_onehot.groupby('Neighborhood').mean().reset_index()
city_grouped

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Volleyball Court,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,'T EILANDJE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0
1,'T FORT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,'T LAAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,'T MESTPUTTEKE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,ALBERTPARK (OOSTWIJK),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,ZANDVLIET-KERN(ZANDVL.-5 DIST),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
250,ZILVERBEEK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
251,ZURENBORG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
252,ZWAANTJES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [296]:
num_top_venues = 5

for hood in city_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = city_grouped[city_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----'T EILANDJE----
              venue  freq
0       Coffee Shop  0.17
1      Cocktail Bar  0.11
2  Sushi Restaurant  0.06
3       Event Space  0.06
4    Sandwich Place  0.06


----'T FORT----
          venue  freq
0         Trail  0.33
1  Cocktail Bar  0.33
2          Park  0.33
3   Opera House  0.00
4        Office  0.00


----'T LAAR----
                venue  freq
0  Athletics & Sports  0.25
1                 Bar  0.25
2              Bakery  0.25
3             Dog Run  0.25
4   Accessories Store  0.00


----'T MESTPUTTEKE----
              venue  freq
0  Greek Restaurant  0.14
1    Ice Cream Shop  0.14
2       Pizza Place  0.14
3              Pool  0.14
4            Bakery  0.14


----ALBERTPARK (OOSTWIJK)----
          venue  freq
0          Park  0.25
1   Coffee Shop  0.12
2       Dog Run  0.12
3        Notary  0.12
4  Tram Station  0.12


----AN-HAVEN/VORMINGSSTATION----
           venue  freq
0  Train Station  0.67
1       Platform  0.33
2  National Park  0.00
3         Office

In [297]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [298]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = city_grouped['Neighborhood']

for ind in np.arange(city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,'T EILANDJE,Coffee Shop,Cocktail Bar,Gastropub,Supermarket,Event Space,Nightclub,Café,Restaurant,Bistro,Pub
1,'T FORT,Trail,Park,Cocktail Bar,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
2,'T LAAR,Dog Run,Bar,Bakery,Athletics & Sports,Event Space,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market
3,'T MESTPUTTEKE,Greek Restaurant,Ice Cream Shop,Pizza Place,Pool,Bakery,Garden Center,Doner Restaurant,Fast Food Restaurant,Falafel Restaurant,Farm
4,ALBERTPARK (OOSTWIJK),Park,Notary,Supermarket,Coffee Shop,Tram Station,Dog Run,Gym,Farmers Market,Ethiopian Restaurant,Event Space


In [299]:
city_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Volleyball Court,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,'T EILANDJE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0
1,'T FORT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,'T LAAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,'T MESTPUTTEKE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,ALBERTPARK (OOSTWIJK),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [315]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 8

city_grouped_clustering = city_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([2, 7, 4, 2, 1, 2, 2, 2, 2, 2, 3, 1, 2, 6, 3, 2, 2, 2, 2, 2, 2, 2,
       2, 6, 2, 7, 1, 7, 2, 4, 2, 2, 2, 2, 2, 0, 2, 4, 1, 6, 2, 1, 6, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 1, 1, 6, 1, 2, 2, 6, 2, 1, 2, 2, 2, 7, 1,
       1, 1, 2, 1, 2, 1, 7, 1, 2, 2, 2, 2, 4, 6, 2, 1, 2, 2, 4, 2, 6, 1,
       2, 2, 2, 7, 6, 2, 1, 6, 5, 6, 2, 2, 6, 2, 2, 2, 3, 4, 2, 0, 2, 2,
       2, 5, 6, 2, 2, 2, 6, 2, 2, 2, 2, 2, 5, 1, 2, 6, 2, 4, 1, 2, 2, 1,
       5, 2, 4, 2, 5, 2, 3, 2, 2, 5, 4, 2, 2, 2, 2, 1, 6, 2, 4, 1, 2, 2,
       6, 6, 2, 1, 2, 6, 2, 2, 2, 2, 7, 1, 2, 2, 2, 6, 2, 4, 6, 2, 1, 1,
       2, 4, 2, 2, 6, 2, 3, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 6, 2, 1, 2,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 7, 2, 2, 6, 1, 2, 2, 3, 5, 1, 2, 6, 1,
       2, 2, 1, 1, 4, 2, 6, 2, 6, 4, 6, 4, 1, 1, 2, 2, 0, 2, 2, 2, 2, 1,
       3, 2, 1, 2, 2, 5, 2, 2, 2, 2, 1, 2])

In [316]:
# to be used if we rerun the below
neighborhoods_venues_sorted.drop(columns=['Cluster Labels'], inplace=True)

In [317]:
# add clustering labels --> Manually change Cluster Labels name to make it work each iteration
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

city_merged = Antw

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
city_merged = city_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Name')

city_merged.head() # check the last columns!

,Code1,Name,Code2,Name2,Municipality,Area_ha,Peri_m,Density,Population,Lon,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1100212MQ,STABROEK,110021,WIJZIGING VAN GEMEENTEGRENS,Antwerpen,81.007781,5228.0,0,0,4.341878,...,Port,Harbor / Marina,Zoo Exhibit,Fish & Chips Shop,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market
1,11002A00-,ANTWERPEN KERN - OUDE STAD (SPAANSE WALLEN ),11002A,1-2-3-4 ADMINISTR. WIJK OF DISTRICT,Antwerpen,22.257464,2208.0,125,2788,4.400889,...,Belgian Restaurant,Italian Restaurant,Restaurant,Coffee Shop,Plaza,Gay Bar,Church,Café,Bookstore,Thai Restaurant
2,11002A01-,KLAPDORP - BROUWERSVLIET,11002A,1-2-3-4 ADMINISTR. WIJK OF DISTRICT,Antwerpen,13.816001,1753.0,115,1595,4.403601,...,Nightclub,Gay Bar,Bar,Coffee Shop,Health & Beauty Service,Greek Restaurant,Café,Snack Place,Empanada Restaurant,Plaza
3,11002A02-,GROENPLAATS (SPAANSE WALLEN),11002A,1-2-3-4 ADMINISTR. WIJK OF DISTRICT,Antwerpen,10.901108,1343.0,49,537,4.402588,...,Coffee Shop,Bar,Juice Bar,Cocktail Bar,Bookstore,Boutique,Beer Bar,Gaming Cafe,Sushi Restaurant,Frozen Yogurt Shop
4,11002A03-,HOOGSTRAAT (SPAANSE WALLEN),11002A,1-2-3-4 ADMINISTR. WIJK OF DISTRICT,Antwerpen,9.244515,1212.0,114,1053,4.398159,...,Bar,Cocktail Bar,Restaurant,Ice Cream Shop,Beer Bar,Bistro,Wine Bar,Coffee Shop,French Restaurant,Hotel


In [318]:
city_merged_nonan = city_merged.dropna(subset=['Cluster Labels'])

In [319]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_merged_nonan['Lat'], city_merged_nonan['Lon'], city_merged_nonan['Name'], city_merged_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [214]:
city_merged_nonan.loc[city_merged_nonan['Cluster Labels'] == 3, city_merged_nonan.columns[[1] + list(range(5, city_merged_nonan.shape[1]))]]

,Name,Area_ha,Peri_m,Density,Population,Lon,Lat,Geometry,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
147,EK-EKERSE PUTTEN,219.689868,9105.0,0,0,4.399198,51.282581,POINT (4.39920 51.28258),3.0,Park,Zoo Exhibit,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Factory


### Discussion
section where you discuss any observations you noted and any recommendations you can make based on the results.

### Conclusion
where you conclude the report.

## Question 1

First, we import BeautifulSoup and Request to import the data from the wiki page

In [21]:
import bs4 as bs
import urllib.request

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

In [22]:
soup = bs.BeautifulSoup(source,'lxml')

The following code is copied from the 'hint' page

In [26]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Now we take a look at the df

In [27]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [29]:
df.shape

(103, 3)

## Question 2

I can't get it work so I used the csv as suggested

In [38]:
data = pd.read_csv('Geospatial_Coordinates.csv')

In [39]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
data.shape

(103, 3)

Merged both dataframes, but they have different name
Check for pandas methods https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#joining-key-columns-on-an-index

In [54]:
dfm = df.merge(data, left_on='PostalCode', right_on='Postal Code')

Check that the merge is done properly and both columns with Postal Code match

In [56]:
dfm

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",M8Y,43.636258,-79.498509


Drop the second Postal Code column

In [224]:
dfm.drop(['Postal Code'], axis=1, inplace=True)

In [225]:
dfm.shape

(103, 5)

## Question 3

Let's import all stuff

In [227]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Let's start by finding out the coordinates for Toronto

In [63]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


We create the map with the neighbourhoods on top. Added also the postal code which was not part of the NY example

In [228]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, PostalCode, borough, Neighborhood in zip(dfm['Latitude'], dfm['Longitude'], dfm['PostalCode'], dfm['Borough'], dfm['Neighborhood']):
    label = '{}, {}, {}'.format(PostalCode, Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [84]:
CLIENT_ID = 'G4CN4VUVBD0EFCCT0LXPFSLWKJAHXJT2BPMPOD235BCMQ1N4' # your Foursquare ID
CLIENT_SECRET = 'VFXVULBUNRTXAHD20FW3A3ZPMHU1V25BJPWLDLH5WDZF4BFB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [198]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [229]:
toronto_venues = getNearbyVenues(names=dfm['Neighborhood'],
                                   latitudes=dfm['Latitude'],
                                   longitudes=dfm['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

The were some venues (4) categorized as 'Neighborhood' so it was messing up everything. The following deletes those venues

In [230]:
toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Neighborhood']
print(toronto_venues.shape)


(2115, 7)


In [231]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",28,28,28,28,28,28
Berczy Park,60,60,60,60,60,60
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14


In [232]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


In [233]:
#test
toronto_venues[['Venue Category']]

,Venue Category
0,Park
1,Fast Food Restaurant
2,Food & Drink Shop
3,Hockey Arena
4,Portuguese Restaurant
5,Coffee Shop
6,Intersection
7,Pizza Place
8,Bakery
9,Coffee Shop


Now we analyse

In [234]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [235]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [236]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [237]:
toronto_grouped.shape

(100, 269)

In [238]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4              Metro Station  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2    Skating Rink  0.14
3  Sandwich Place  0.14
4     Coffee Shop  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop  0.09
1           Bank  0.09
2       Pharmacy  0.05
3          Diner  0.05
4  Shopping Mall  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4  Moroccan Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0          Restaurant  0.07
1  Italian Restaurant  0.07
2         Coffee Shop  0.07
3      Sandwich Place  0.07
4             B

We put it into Pandas

In [239]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

With the top 10

In [240]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Pub,Sandwich Place,Gym,Airport Lounge,Event Space,Escape Room,Electronics Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant,Supermarket,Sushi Restaurant,Ice Cream Shop,Shopping Mall,Middle Eastern Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Restaurant,Coffee Shop,Italian Restaurant,Juice Bar,Boutique,Japanese Restaurant,Liquor Store,Pharmacy,Spa


And finally, we cluster in 5

In [241]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0])

In [242]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfm

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Fast Food Restaurant,Food & Drink Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,Intersection,Coffee Shop,Pizza Place,Portuguese Restaurant,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Pub,Bakery,Park,Café,Restaurant,Breakfast Spot,Theater,Yoga Studio,Electronics Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Carpet Store,Miscellaneous Shop,Vietnamese Restaurant,Boutique,Dumpling Restaurant,Eastern European Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Italian Restaurant,Bank,Bar,Café,Diner,Discount Store,Beer Bar


Droping NAN to avoid problems

In [250]:
toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

In [252]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighborhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The following code is to explore each cluster. Only one showed for now

In [257]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,2.0,Park,Bar,Women's Store,Colombian Restaurant,Comfort Food Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
35,East York/East Toronto,2.0,Park,Intersection,Convenience Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dessert Shop,Donut Shop
40,North York,2.0,Park,Business Service,Airport,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
52,North York,2.0,Park,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
64,York,2.0,Park,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
66,North York,2.0,Park,Convenience Store,Construction & Landscaping,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store,Escape Room
91,Downtown Toronto,2.0,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
101,Etobicoke,2.0,Park,Baseball Field,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Event Space
